In [1]:
#%pip install liwc

In [2]:
import nltk
import string
import re
from collections import Counter
import pandas as pd
import spacy
from itertools import compress
import numpy as np
import liwc
parse, category_names = liwc.load_token_parser('data/dict_r.dic')

In [3]:
#!python -m spacy download es_core_news_sm
#nltk.download('punkt')
#nltk.download('stopwords')

In [4]:
nlp = spacy.load('es_core_news_sm')

In [5]:
institutions = pd.read_excel('data/ELA - Instituciones.xlsx')

In [6]:
def preprocess(df, concept = None, all_less_concept = None):
  institutions = df
  if concept is not None:
    institutions = institutions[institutions["categoria"] == concept]

  if all_less_concept is not None:
    institutions = institutions[institutions["categoria"] != all_less_concept]

  institutions['fundamento'] = institutions['fundamento'].str.lower()
  #institutions['fundamento'] = institutions['fundamento'].apply(lambda x: re.sub("[0-9]", '', x))
  institutions['fundamento'] = institutions['fundamento'].str.translate(str.maketrans(' ', ' ', string.punctuation))
  institutions['fundamento'] = institutions['fundamento'].str.strip()
  institutions['fundamento'] = institutions['fundamento'].str.replace('\n', ' ')
  institutions['fundamento'] = institutions['fundamento'].str.replace('  ', ' ')
  institutions['fundamento'] = institutions['fundamento'].str.replace('   ', ' ')
  institutions['fundamento'] = institutions['fundamento'].str.replace('    ', ' ')
  institutions['fundamento'] = institutions['fundamento'].str.replace('     ', ' ')

  institutions = institutions[[len(str(x).split()) > 2 for x in institutions.fundamento]]
  return institutions.reset_index(drop=True)

In [7]:
institutions_ffaa = preprocess(institutions, "Fuerzas Armadas")

/var/folders/h_/6z9y83sx7pnf2t3_ym4sx6_00000gn/T/ipykernel_1062/1406821845.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  institutions['fundamento'] = institutions['fundamento'].str.lower()
/var/folders/h_/6z9y83sx7pnf2t3_ym4sx6_00000gn/T/ipykernel_1062/1406821845.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  institutions['fundamento'] = institutions['fundamento'].str.translate(str.maketrans(' ', ' ', string.punctuation))
/var/folders/h_/6z9y83sx7pnf2t3_ym4sx6_00000gn/T/ipykernel_1062/1406821845

In [8]:
institutions_less_ffaa = preprocess(institutions, all_less_concept = "Fuerzas Armadas")

/var/folders/h_/6z9y83sx7pnf2t3_ym4sx6_00000gn/T/ipykernel_1062/1406821845.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  institutions['fundamento'] = institutions['fundamento'].str.lower()
/var/folders/h_/6z9y83sx7pnf2t3_ym4sx6_00000gn/T/ipykernel_1062/1406821845.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  institutions['fundamento'] = institutions['fundamento'].str.translate(str.maketrans(' ', ' ', string.punctuation))
/var/folders/h_/6z9y83sx7pnf2t3_ym4sx6_00000gn/T/ipykernel_1062/1406821845

In [9]:
institutions_all = preprocess(institutions)

### 1 - Diccionario

In [10]:
%%time

df = pd.DataFrame()
l_derechos = []

for ix in range(institutions_ffaa.shape[0]):
  texto = str(institutions_ffaa.fundamento[ix]).lower()
  doc = nlp(texto)
  tokens = [word.orth_ for word in doc]
  tcounts1 = Counter(category for token in tokens for category in parse(token))
  palabras_derechos = list(compress(tokens, ['derechos' in parse(token) for token in tokens]))
  df_counts = pd.DataFrame.from_dict([tcounts1]).reset_index()
  df_counts['index'] = ix
  df_counts['texto'] = texto
  df_counts['word_derechos'] =  ['' if len(palabras_derechos)==0 else palabras_derechos]
  df = pd.concat([df,df_counts], axis=0, ignore_index=True)

  if len(palabras_derechos)>0:
        l_derechos.extend(palabras_derechos)

CPU times: user 31.9 s, sys: 235 ms, total: 32.2 s
Wall time: 32.2 s


In [11]:
df_concept_ffaa = pd.DataFrame.from_dict(Counter(l_derechos), orient = 'index').reset_index()
df_concept_ffaa = df_concept_ffaa.rename(columns = {'index' : 'term', 0 : 'count'})
df_concept_ffaa.sort_values('count', ascending = False)[0:20]

,term,count
0,proteger,73
2,defender,58
9,respetar,25
8,cumplir,21
13,decir,15
7,ejercer,13
5,contribuir,12
3,responder,9
11,trabajar,5
12,procurar,5


### 2 - Verificacion diccionario

In [12]:
df_concept_ffaa

,term,count
0,proteger,73
1,apoyar,4
2,defender,58
3,responder,9
4,usar,4
5,contribuir,12
6,vivir,1
7,ejercer,13
8,cumplir,21
9,respetar,25


### 3 - Desempeño

In [169]:
df_dummy_categories_ffaa = pd.get_dummies(df_concept_ffaa.groupby(by = ['term']).count().reset_index()['term'])

In [170]:
df_dummy_categories_ffaa = df_dummy_categories_ffaa[0:0]

In [171]:
df_institutions_all_group = institutions_all.groupby(by = ['categoria']).count().reset_index()['categoria']
df_dummy_categories_ffaa['categoria'] = df_institutions_all_group

In [172]:
df_dummy_categories_ffaa = df_dummy_categories_ffaa.set_index("categoria")

for col in df_dummy_categories_ffaa.columns:
    df_dummy_categories_ffaa[col].values[:] = 0

df_dummy_categories_ffaa

,apoyar,conocer,conservar,contribuir,cumplir,decir,defender,educar,ejercer,guardar,procurar,proteger,respetar,responder,servir,tomar,trabajar,usar,vivir
categoria,,,,,,,,,,,,,,,,,,,
Asamblea Constituyente,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Asambleas y Cabildos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Autonomía Banco Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Autonomía regional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cambio o reforma constitucional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Carabineros,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Congreso o Parlamento Unicameral,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Congreso o parlamento,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Contraloría general / Tribunales de cuentas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [173]:
for index, row in institutions_all.iterrows():
  for i, v in df_dummy_categories_ffaa.loc[row['categoria']].items():
    df_dummy_categories_ffaa.loc[row['categoria']][i] = df_dummy_categories_ffaa.loc[row['categoria']][i] + row['fundamento'].count(i)


In [174]:
df_dummy_categories_ffaa

,apoyar,conocer,conservar,contribuir,cumplir,decir,defender,educar,ejercer,guardar,procurar,proteger,respetar,responder,servir,tomar,trabajar,usar,vivir
categoria,,,,,,,,,,,,,,,,,,,
Asamblea Constituyente,0.0,2.0,0.0,0.0,4.0,4.0,0.0,0.0,7.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,2.0,0.0,2.0
Asambleas y Cabildos,2.0,2.0,0.0,0.0,1.0,0.0,0.0,8.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Autonomía Banco Central,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Autonomía regional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
Cambio o reforma constitucional,0.0,6.0,3.0,2.0,4.0,5.0,2.0,1.0,9.0,4.0,0.0,2.0,9.0,8.0,0.0,1.0,2.0,0.0,6.0
Carabineros,0.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Congreso o Parlamento Unicameral,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0
Congreso o parlamento,2.0,10.0,5.0,0.0,33.0,11.0,2.0,0.0,21.0,33.0,1.0,5.0,7.0,5.0,10.0,15.0,19.0,3.0,5.0
Contraloría general / Tribunales de cuentas,2.0,3.0,0.0,0.0,18.0,3.0,1.0,0.0,25.0,16.0,0.0,6.0,3.0,2.0,1.0,6.0,1.0,1.0,0.0
